In [43]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-254578
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-254578


In [25]:
#from azureml.core import Workspace, Experiment

#ws = Workspace.get(
#    name='quick-starts-ws-254578',
#    subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f',
#    resource_group='aml-quickstarts-254578',
#)
#exp = Experiment(workspace=ws, name="udacity-project")


In [44]:
from azureml.core import Workspace, Experiment

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-254578
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-254578


In [47]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cluster-exerc"


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

compute_comfig = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
aml_compute = ComputeTarget.create(ws, cluster_name, compute_comfig)
#aml_Compute.wait_for_completion(show_output=True)

In [51]:
from azureml.widgets import RunDetails
from azureml.core import Workspace,Experiment,Run
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling({'learning_rate':choice(0.0001, 0.00001)})
# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, slack_amount=None, delay_evaluation=0)
if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#src = ### YOUR CODE HERE ###
src = ScriptRunConfig(source_directory='.', script='train.py', compute_target=aml_compute)
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='validation_acc',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 4,
                                     max_concurrent_runs=4)

In [54]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
#hyperdrive_run.wait_for_completion()
RunDetails(hyperdrive_run)

In [58]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
list(exp.get_runs())
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print('Best Run ID:', best_run.id)
print('test')
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n lr:', parameter_values[3])
print('keep probability:', parameter_Values[5])

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=4)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = experiment.submit(automl_config, show_output = True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###